In [1]:
import nltk
import re
import csv
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import datefinder
from datetime import datetime
import MySQLdb
import MySQLdb.cursors
import requests

class experience_details():
    
    def search_keyword(self,text, keyword_list):
        for word in keyword_list:
            word = str(word)
            if word.title() in text or word.upper() in text or word.capitalize() in text:
                return True
        return False
    
    def load_work_segment(self,pdf_to_text_list,education_keywords,education_degree_category,
                               project_keywords, work_experience_keywords,other_keywords,
                              work_segment):#skill_keywords,
        
        text_list = pdf_to_text_list
        def validate_text(text):
            
            user_segment = []

            # check email in name object
            if text.find("@") > 0 or text.find("github.com") >= 0 or text.find("linkedin.com") >= 0:
                user_segment.append(text)
                #print ("Email found")
                return False

            # check phone number in name object
            cell = re.search("\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}",
                             text)
            if cell:
                #print ("Cell found")
                user_segment.append(text)
                return False

            # check hyperlinks in name object
            urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
            if len(urls) > 0:
                #print ("link found")
                user_segment.append(text)
                return False

            if text.find("GPA") > 0:
                #print ("GPA found")
                education_segment.append(text)
                return False

            return True
        for i,text in enumerate(text_list):
            #print(i,"=====",text)
            #text = text.replace(":","")
            text = text.replace("|","")
            text = re.sub(r'[^\w\s]','',text)
            #text = text.encode('ascii', 'ignore')
            
            flag = False
            if self.search_keyword(text, work_experience_keywords):
                #print(text)
                work_segment.append(text)
                i += 1
                #print("Next Line",i)
                flag = True
                try:
                    while True:
                        text = text_list[i]
                        text = re.sub(r'[^\w\s]','',text)
                        text = text.replace("'","")
                        text = text.replace("|","")
                        #text = text.encode('ascii', 'ignore')
                        if not self.search_keyword(text, education_keywords) and not self.search_keyword(text,
                                                                                              education_degree_category) and not self.search_keyword(
                                text, project_keywords) and not self.search_keyword(text, other_keywords):#and not self.search_keyword(text, skill_keywords)

                            #if validate_text(text):
                            work_segment.append(text)
                        else:
                            break
                        i += 1
                except:
                    print("last line")
            if flag:
                break
                
        #print(work_segment)
        return work_segment
    
    
    "===== extract work experience ============"
    
    def extract_work_experience(self,work_experience_keywords,work_seg,work_experience,infoDict):
    
        cnx = MySQLdb.connect(host = 'localhost',
                                 database = 'cv_parse',
                                 user = 'root',
                                 password = 'root', 
                                 compress = 1,
                                 cursorclass=MySQLdb.cursors.DictCursor)

        cursor = cnx.cursor()
        
        work_segment = work_seg
        work_experience = work_experience
        infoDict = infoDict
        infoDict['work_keyword'] = []

        def extract_exp_title_first(text,title_flag,date_flag,skill_flag):
            closest_title = ''
            closest_company = ''
            ty = {}
            max_score, longest_length = 0, 0
            if not title_flag:
                text = text.replace("'","")
                query = """SELECT position_title FROM cv_parse.position_title_table
                            WHERE '"""+ text + """' LIKE CONCAT('%',position_title,'%')"""
                #print(query)
                cursor.execute(query)
                tit = cursor.fetchall()
                #print("After query title ===", tit)
                if tit:
                    #print("Inserted text ==== ",text)
                    title = tit[0]['position_title']
                    if (re.match('[?.!]$', title) is None):
                        closest_title = title
                    #print(closest_title)
                        title_flag = True

                    score = fuzz.ratio(str(title).lower(), text.lower())
                    #print (university, score)
                    if score > 90 and max_score < score:
                        max_score = score
                        if (re.match('[?.!]$', title) is None):
                            closest_title = title
                            title_flag = True
                        #print(closest_title)

                    if title_flag:
                        work_experience['Job_Title'].append(closest_title)

            if not date_flag:
                #print("==== again date ====", date_flag)
                matches = re.findall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{,2}[a-z]*(?:-|,|\s)?\s?\d{2,4})',text)
                #print(matches)
                if matches:
                    #print(len(matches))
                    if len(matches) == 1:
                        ty = {'start_year':matches[0],'end_year':"Present"}
                    else:
                        ty = {'start_year':matches[0],'end_year':matches[1]}
                    work_experience['Date'].append(ty)
                    date_flag = True
                else:
                    matches = re.findall(r'\b\d{4}\b',text)
                    if matches:
                        if len(matches) == 1:
                            ty = {'start_year':matches[0],'end_year':"Present"}
                        else:
                            ty = {'start_year':matches[0],'end_year':matches[1]}

                        work_experience['Date'].append(ty)
                        date_flag = True
            if not skill_flag:
                text = text.replace("'","")
                query = """SELECT skills FROM cv_parse.skill_table
                            WHERE '"""+ text + """' LIKE CONCAT('% ',skills,' %')"""

                cursor.execute(query)
                sk = cursor.fetchall()

                if sk:
                    skill = sk[0]['skills']
                    #print("parse skill =====",skill)
                    sl.append(skill)
                    skill_flag = True

            return title_flag,date_flag,skill_flag

        noun_phrases = []
        closest_company_1, closest_company_2 = "", ""
        comp = []
        pos_tit = []
        work_experience['Date'] = []
        work_experience['Job_Title'] = []
        work_experience['company'] = []
        sl = []
        work_experience['skills'] = []
        #work_experience['keywords'] = []
        max_score, longest_length = 0, 0
        compan=[]
        title_match=[]
        date=[]
        

        for i in range(0,len(work_segment)):

            closest_title = ''
            closest_company = ''
            ty = {}
            company = ''
            title = ''

            title_flag = False
            company_flag = False
            date_flag = False
            skill_flag = False
            #print(len(work_segment))
            text = work_segment[i]
            #print(text)
            #print(i)
            if not company_flag:

                text = text.replace("'","")
                text = text.replace("Ltd","Limited")
                text = text.replace("ltd","Limited")
                
                #print("Text before query===", text)
                query2 = """SELECT company_name FROM cv_parse.company_name_table
                            WHERE '"""+ text + """' LIKE CONCAT('%',company_name,'%')"""
                #print(query2)
                cursor.execute(query2)
                comp = cursor.fetchall()
                #print("After query title ===", comp)
                if comp:

                    #print("text after find company",text)
                    company = comp[0]['company_name']

                    #print("company name",company)

                    if re.match(r'[a-zA-Z0-9_].',company) is not None:
                        longest_length = len(company)
                        closest_company_1 = company
                        #print(closest_company_1,"==== 1")
                        company_flag = True
                #elif not closest_company_1:


                score = fuzz.ratio(str(company).lower(), text.lower())
                #print (university, score)
                if score > 90 and max_score < score and not text == " ":
                    #print(company,score)
                    max_score = score
                    if re.match(r'[a-zA-Z0-9_].',company) is not None:
                        closest_company_2 = company
                        company_flag = True
                        #print(closest_company_2,"===2")

                if company_flag:
                    if len(closest_company_1) > len(closest_company_2):
                        closest_company = closest_company_1
                    else:
                        closest_company = closest_company_2

                    if closest_company:
                        #print("Company_name ========",closest_company)
                        work_experience['company'].append(closest_company)

                    #print(i)
                    j = i
                    #print(j)

                    while (True):
                        if not title_flag:
                            text = text.replace("'","")
                            query = """SELECT position_title FROM cv_parse.position_title_table
                                    WHERE '"""+ text + """' LIKE CONCAT('%',position_title,'%')"""
                            cursor.execute(query)
                            tit = cursor.fetchall()

                            if tit:# and longest_length <= len(title):
                                title = tit[0]['position_title']
                                #longest_length = len(title)
                                closest_title = title
                                #print(closest_title)
                                title_flag = True
                                
                                score = fuzz.ratio(str(title).lower(), text.lower())
                                #print (university, score)
                                if score > 90 and max_score < score:
                                    max_score = score
                                    if re.match(r'[a-zA-Z0-9_].',title) is not None:
                                        closest_title = title
                                        title_flag = True
                                    #print("After company name ======",closest_title)

                            if title_flag:
                                if closest_title != '' or re.search(r'^[a-zA-Z]{1}$',closest_title) is None:
                                    work_experience['Job_Title'].append(closest_title)

                        if not date_flag:
                            matches = re.findall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{,2}[a-z]*(?:-|,|\s)?\s?\d{2,4})',text)
                            #print(matches)
                            if matches:
                                #print(len(matches))
                                if len(matches) == 1:
                                    ty = {'start_year':matches[0],'end_year':"Present"}
                                else:
                                    ty = {'start_year':matches[0],'end_year':matches[1]}
                                work_experience['Date'].append(ty)
                                date_flag = True
                            else:
                                matches = re.findall(r'\b\d{4}\b',text)
                                if matches:
                                    if len(matches) == 1:
                                        ty = {'start_year':matches[0],'end_year':"Present"}
                                    else:
                                        ty = {'start_year':matches[0],'end_year':matches[1]}

                                    work_experience['Date'].append(ty)
                                    date_flag = True
                                    
                        if not skill_flag:
                            text = text.replace("'","")
                            query = """SELECT skills FROM cv_parse.skill_table
                                        WHERE '"""+ text + """' LIKE CONCAT('% ',skills,' %')"""

                            cursor.execute(query)
                            sk = cursor.fetchall()

                            if sk:
                                skill = sk[0]['skills']
                                #print("parse skill =====",skill)
                                sl.append(skill)
                                skill_flag = True

 
                        if title_flag and date_flag and skill_flag or i >= len(work_segment):
                            break

                        i+=1
                        #print("Title_flag---", title_flag)
                        #print("Date_flag---", date_flag)

                        if not i >= len(work_segment):
                            text = work_segment[i]
                        else:
                            #print("Check in next lines")
                            k = 0
                            #print("while loop === ",j)

                            while (k<j):
                                text = work_segment[k]
                                #print(k,"second while loop=========", text)

                                if not title_flag:
                                    text = text.replace("'","")
                                    query = """SELECT position_title FROM cv_parse.position_title_table
                                            WHERE '"""+ text + """' LIKE CONCAT('%',position_title,'%')"""
                                    cursor.execute(query)
                                    tit = cursor.fetchall()

                                    if tit:# and longest_length <= len(title):
                                        title = tit[0]['position_title']
                                        #longest_length = len(title)
                                        closest_title = title
                                        #print(closest_title)
                                        title_flag = True
                                        score = fuzz.ratio(str(title).lower(), text.lower())
                                        #print (university, score)
                                        if score > 90 and max_score < score:
                                            max_score = score
                                            if re.match(r'[a-zA-Z0-9_].',title) is not None:
                                                closest_title = title
                                                title_flag = True
                                            #print(closest_title)

                                    if title_flag:
                                        if closest_title != '' or re.search(r'^[a-zA-Z]{1}$',closest_title) is None:
                                            work_experience['Job_Title'].append(closest_title)

                                if not date_flag:
                                    matches = re.findall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{,2}[a-z]*(?:-|,|\s)?\s?\d{2,4})',text)
                                    #print(matches)
                                    if matches:
                                        #print(len(matches))
                                        if len(matches) == 1:
                                            ty = {'start_year':matches[0],'end_year':"Present"}
                                        else:
                                            ty = {'start_year':matches[0],'end_year':matches[1]}
                                        work_experience['Date'].append(ty)
                                        date_flag = True
                                    else:
                                        matches = re.findall(r'\b\d{4}\b',text)
                                        if matches:
                                            if len(matches) == 1:
                                                ty = {'start_year':matches[0],'end_year':"Present"}
                                            else:
                                                ty = {'start_year':matches[0],'end_year':matches[1]}

                                            work_experience['Date'].append(ty)
                                            date_flag = True
                                            
                                if not skill_flag:
                                    text = text.replace("'","")
                                    query = """SELECT skills FROM cv_parse.skill_table
                                                WHERE '"""+ text + """' LIKE CONCAT('% ',skills,' %')"""

                                    cursor.execute(query)
                                    sk = cursor.fetchall()

                                    if sk:
                                        skill = sk[0]['skills']
                                        #print("parse skill =====",skill)
                                        sl.append(skill)
                                        skill_flag = True

                                k+=1
                                #print("ELSE Title_flag---", title_flag)
                                #print("ELSE Date_flag---", date_flag)
                                #print("=============================")
                            break
                else:
                    #print("----- Company name not in Sentence -----")
                    title_flag,date_flag,skill_flag = extract_exp_title_first(text,title_flag,date_flag,skill_flag)
                    #print("Title_flag---", title_flag)
                    #print("Date_flag---", date_flag)

                if not title_flag or not date_flag or not company_flag:
                            infoDict['work_keyword'].append(text)
        work_experience['skills'] = list(set(sl))
        e = work_experience
        return e, infoDict
    
    
    
    #def extract_work_experience(self,work_experience_keywords,work_seg,work_experience,infoDict):
        

/Users/imac086/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
experience_details()

In [1]:
s = ()
if s:
    print(true)

In [1]:
import re
s = "string With. Punctuation?"
s = re.sub(r'[^\w\s]','',s)

In [2]:
print(s)

string With Punctuation


In [11]:
import csv
def read_csv(input_file):
    file = open(input_file, 'r')
    reader = csv.reader(file)
    return reader

def get_keywords(dir_path,file_name):
    reader = read_csv(dir_path + file_name)
    keywords = []
    for row in reader:
        keywords.append(row[0])
    return keywords
def search_keyword(text, keyword_list):
    for word in keyword_list:
        word = str(word)
        if word.title() in text or word.upper() in text or word.capitalize() in text:
            return True
    return False

In [13]:
dir_path = '/Users/imac086/Documents/premal/projects_Quix/CV_parsing/final_cv_parsing_code/database/'
work_experience_keywords = get_keywords(dir_path,"work_experience_segment.csv")
if search_keyword("CAREER HISTORY",work_experience_keywords):
    print('CAREER HISTORY')

CAREER HISTORY


In [1]:
        #try:
        for text in (work_segment):
            lines = nltk.word_tokenize(text)
            tags = nltk.pos_tag(lines)

            nouns = [word for word,pos in tags if(pos == "NN")]
            company = " ".join(nouns)
            comp.append(company)


            title = [word for word,pos in tags if(pos == "NNP")]
            ti = " ".join(title)
            pos_tit.append(ti)

NameError: name 'work_segment' is not defined